# LangChaing for LLM applications development Course

### LangChain: Models, Prompts and Output Parsers

Outline:

*  Direct API calls to OpenAI
*  API calls through LangChain:
    *  Prompts
    *  Models
    *  Output parsers

Get your OpenAI API Key

In [5]:
#!pip install python-dotenv
#!pip install openai

In [6]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

from langchain_openai import AzureChatOpenAI


API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
API_BASE = os.getenv('AZURE_OPENAI_ENDPOINT_GPT')
#------------------------
API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [7]:
client = AzureOpenAI(
    azure_endpoint= API_BASE,
    api_version = '2024-02-15-preview',
    api_key = API_KEY
)

def get_completion(prompt):
    messages = [
                {
                    "role": "user", 
                    "content": prompt
                }
               ]
    response = client.chat.completions.create(
        model = 'gpt-4o',
        frequency_penalty=0.1,
        temperature=0,
        seed=5,
        max_tokens=4092,
        messages = messages
    )
    
    return response.choices[0].message.content


In [8]:
get_completion("Cuantas champions tiene el madrid?")

'Hasta la fecha de corte de mi conocimiento en octubre de 2023, el Real Madrid ha ganado 14 títulos de la UEFA Champions League.'

# Using LLM without LangChaing

For this example we are gonna change a message written in English pirate to a more formal lenguage

In [50]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [51]:
response = get_completion(prompt)
response

"I'm quite upset that my blender lid came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now."

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [3]:
#!pip install --upgrade langchain

### Model

In [ ]:
from langchain_openai import AzureChatOpenAI

In [35]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
llm_model = 'gpt-4o'
chat = AzureChatOpenAI(temperature=0.0, azure_deployment=llm_model,api_version='2024-02-15-preview')
chat

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D0FF615BA0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D0FF6211D0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x000001D0FF60D950>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000001D0FF60DA90>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://advanced-prompting.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview', deployment_name='gpt-4o', openai_api_version='2024-02-15-preview', openai_api_type='azure')

### Prompt template

In [36]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

To repeatedly use the template, we can import ChatPromptTemplate

In [37]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [38]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [39]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [ ]:
# Style, which we want our sentence to be transformed
customer_style = """American English \
in a calm and respectful tone
"""

#Text, text we wanna transform to a specific style..
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
# The custom message will generate the prompt and send it to a LLM model
# to get a respond

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [42]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [43]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [52]:
# Call the LLM to translate to the style of the customer message
# We send the promt created, through 
customer_response = chat(customer_messages)

In [53]:
print(customer_response.content)

I'm quite upset that my blender lid came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now.


Now we want to transform a service reply from a polite tone to a pirate tone

In [54]:
#Text
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

#Style
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

the cool thing is , that we can reuse the template created before...

In [55]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [56]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy, me hearty customer! I be regretful to inform ye that the warranty be not coverin' the cleanin' expenses for yer galley, as it be yer own misstep in misusin' the blender by forgettin' to secure the lid afore startin' the contraption. Alas, such be the way of the seas! Fair winds to ye!


Why are we using prompt-templates instead of just an f-string? 

* Promts can be long, when creaiting apliccations
* Prompt templates allows to reuse good prompts
* LangChain has also build in prompts that allows you create some commons operations, such us summarization, Q&A, connecting to a SQL, or different API´s.

One cool thing is that it also supports output parsing (Helps to generate the LLMS outputs in certain format.)


LangChain use Thought, Action and Observation as keywords for Chain-of-Thought Reasoning (ReAct)

* Thoughts: is what the LLM is thinking. By giving them space to think it will generate more accurate responds.

* Action: Carry an specific action

* Observation: Show what the prompt has lean after carrying that specific action.

(With this you can extract the specifict text has been attached to the each project, so you would be able to have a smooth extraction.)

And it also have a Parser to correct interpreter the output the LLM gives


## Output Parsers

Let's start with defining how we would like the LLM output to look like:


For this example we are gonna have a DICT and we are gonna see how the output Parser works

In [ ]:
#Example of desired output

{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [58]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

#The review template asks the LLM to take as input a customer review
# and extracts three fields; gift, delivery_days, price_value and
# format then in a JSON output

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

We ran this without using a Parser, to show the problem.

That we are not gonna receive the output in the format we want (dict)

In [59]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [61]:
messages = prompt_template.format_messages(text=customer_review)
chat = AzureChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```


If we look at the type of the respond is a string, not a dictionary as we want.

In [62]:
type(response.content)

str

In [ ]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string

response.content.get('gift')

### Parse the LLM output string into a Python dictionary

Now we are gonna do it with Parsers

In [66]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [67]:
# We define the schema we want for the outpy, the we introduce the into a list. 

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [68]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

Thanks to the LangChain structure it allows to auto generate a prompt base on the required schema we create.

In [69]:
format_instructions = output_parser.get_format_instructions()

In [70]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


So here it´s the new review templte, and in this template we add at the end

a format instructions where we indicate the schema

In [71]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [72]:
# Let´s take a look into the actual prompt
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [73]:
#We send the prompt to the LLM
response = chat(messages)

In [74]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


We see the output is still a string, so now we will send it to the
Parser that we´ve created before

In [77]:
print(type(response.content))

<class 'str'>


In [78]:
# We parse the output into a dict 

output_dict = output_parser.parse(response.content)

In [79]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

Now we have an anwers in a dict form, instead of str

In [80]:
type(output_dict)

dict

In [81]:
output_dict.get('delivery_days')

'2'